In [1]:
import numpy as np
import pandas as pd
import duckdb
import sqlalchemy
from pandas_gbq import read_gbq

In [2]:
import great_expectations as gx
from great_expectations import expectations as gxe

import pprint
import os

<div class="alert alert-block alert-info">
data testing

In [ ]:
context = gx.get_context()
# query bigquery
project_id = "projectm2-aiess"
query = "SELECT * FROM olist_brazilian_ecommerce_target_star.DIM_CUSTOMERS"
df_customers = read_gbq(query, project_id=project_id)


data_source_name = "olist.dim_customers"
data_source = context.data_sources.add_pandas(name=data_source_name)

Downloading: 100%|██████████|


In [8]:
# create asset
data_asset_name = "olist.dim_customers_asset"
data_asset = data_source.add_dataframe_asset(name=data_asset_name)


In [9]:
# create batch
batch_definition_name = "batch_customers_dataframe"
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

batch_parameters = {"dataframe": df_customers}

new_batch = batch_definition.get_batch(batch_parameters=batch_parameters)

In [10]:
print(new_batch.head(4))

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 241.15it/s]

                    pk_customer_sid                customer_unique_id  \
0  10ad09201fcc1c82d181ff7234bcdb3b  94742cd1fbac9146be7e2a139b63e13c   
1  8a0108267d9258a0ec9f74381bc9b0de  7a2dc4682890550ebe3b8befcea3d55c   
2  cd281c1a7d26cd29a3ed4b029fce7270  086d6b5b5ba195a91aa0a6ec8e75d1a4   
3  31dbc13addc753e210692eacaea065e4  5dbba6c01268a8ad43f79157bf4454a0   

   customer_zip_code_prefix customer_city customer_state  \
0                     69900    rio branco             AC   
1                     69900    rio branco             AC   
2                     69900    rio branco             AC   
3                     69900    rio branco             AC   

                load_date  
0  2025-06-13 01:47:46 PM  
1  2025-06-13 01:47:46 PM  
2  2025-06-13 01:47:46 PM  
3  2025-06-13 01:47:46 PM  


In [11]:
# Create a new suite for all dimension tables schema validation
suite_name = "schema_dim_customers_expectation"
suite = gx.ExpectationSuite(name=suite_name)

schema_dim_customers_expectation = gx.expectations.ExpectColumnToExist(
    column="customer_sid", column_index=0
)

context.suites.add_or_update(suite)
suite.add_expectation(schema_dim_customers_expectation)

definition_name = "schema_dim_customers_definition"
validation_definition = gx.ValidationDefinition(
    data=batch_definition, suite=suite, name=definition_name
)

In [14]:
context = gx.get_context()
# List of GBQ tables and their expected columns with types
gbq_tables_with_columns_and_types = {
    "olist_brazilian_ecommerce_target_star.DIM_PAYMENTS": {"payment_sid": "string"},
    "olist_brazilian_ecommerce_target_star.DIM_REVIEWS": {"review_sid": "string"},
    "olist_brazilian_ecommerce_target_star.DIM_GEOLOCATION": {"geolocation_zip_code_prefix": "string"},
    "olist_brazilian_ecommerce_target_star.DIM_ITEMS": {"item_sid": "string"},
    "olist_brazilian_ecommerce_target_star.DIM_DATE": {"date_sid": "integer"},
}

# Iterate over the list of tables and process each one
for table_name, expected_columns in gbq_tables_with_columns_and_types.items():
    # Query the table from GBQ
    query = f"SELECT * FROM {table_name}"
    df_table = read_gbq(query, project_id="projectm2-aiess")

    # Generate unique names for data source and asset
    data_source_name = f"{table_name}_data_source"
    asset_name = f"{table_name}_asset"

    # Add data source
    data_source = context.data_sources.add_pandas(name=data_source_name)

    # Add DataFrame asset
    data_asset = data_source.add_dataframe_asset(name=asset_name)

    # Add batch definition
    batch_definition_name = table_name
    batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

    # Get the batch and print the first few rows
    batch_parameters = {"dataframe": df_table}
    batch = batch_definition.get_batch(batch_parameters=batch_parameters)
    print(f"Batch for {table_name}:")
    print(batch.head(4))

    # Create Expectation Suite
    suite_name = f"{table_name}_expectation_suite"
    suite = gx.ExpectationSuite(name=suite_name)
    suite = context.suites.add(suite)

    # Add ExpectColumnValuesToBeOfType expectations for each expected column
    for column, column_type in expected_columns.items():
        expectation = gx.expectations.ExpectColumnValuesToBeOfType(
            column=column, type_=column_type
        )
        suite.add_expectation(expectation)

    # Create validation definition
    definition_name = f"{table_name}_validation_definition"
    validation_definition = gx.ValidationDefinition(
        data=batch_definition, suite=suite, name=definition_name
    )

    # Run validation
    validation_results = validation_definition.run(batch_parameters=batch_parameters)
    print(f"Validation results for {table_name}:")
    print(validation_results)

Downloading: 100%|██████████|
Batch for olist_brazilian_ecommerce_target_star.DIM_PAYMENTS:


Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 699.28it/s] 

                     pk_payment_sid  payment_sequential payment_type  \
0  09b457f1152126535a2e7d4a3b73ff6f                   1       boleto   
1  ee0829e4d1cc7547c044b8643c14da38                   1       boleto   
2  db7576b1fe440f4c0a808855aacf0948                   1       boleto   
3  6ef587afa4703fd874edb7ade8efc7b0                   1       boleto   

   payment_installments  payment_value               load_date  
0                     1          14.08  2025-06-13 01:48:07 PM  
1                     1          18.52  2025-06-13 01:48:07 PM  
2                     1          18.62  2025-06-13 01:48:07 PM  
3                     1          20.23  2025-06-13 01:48:07 PM  



Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 628.08it/s] 


Validation results for olist_brazilian_ecommerce_target_star.DIM_PAYMENTS:
{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "column": "payment_sid",
          "type_": "string",
          "batch_id": "olist_brazilian_ecommerce_target_star.DIM_PAYMENTS_data_source-olist_brazilian_ecommerce_target_star.DIM_PAYMENTS_asset"
        },
        "meta": {},
        "id": "a63c3c95-53e3-431e-ab49-9b7cbba99fc5"
      },
      "result": {},
      "meta": {},
      "exception_info": {
        "exception_traceback": "Traceback (most recent call last):\n  File \"/Users/sqa.k/miniconda3/envs/elt/lib/python3.10/site-packages/great_expectations/validator/validator.py\", line 650, in graph_validate\n    result = expectation.metrics_validate(\n  File \"/Users/sqa.k/miniconda3/envs/elt/lib/python3.10/site-packages/great_expectations/expectations/expectation.py\", line 1147, 

...

Validation results for olist_brazilian_ecommerce_target_star.DIM_ITEMS:
{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "column": "item_sid",
          "type_": "string",
          "batch_id": "olist_brazilian_ecommerce_target_star.DIM_ITEMS_data_source-olist_brazilian_ecommerce_target_star.DIM_ITEMS_asset"
        },
        "meta": {},
        "id": "43187fa5-15dd-49f6-acbe-a47528fe4082"
      },
      "result": {},
      "meta": {},
      "exception_info": {
        "exception_traceback": "Traceback (most recent call last):\n  File \"/Users/sqa.k/miniconda3/envs/elt/lib/python3.10/site-packages/great_expectations/validator/validator.py\", line 650, in graph_validate\n    result = expectation.metrics_validate(\n  File \"/Users/sqa.k/miniconda3/envs/elt/lib/python3.10/site-packages/great_expectations/expectations/expectation.py\", line 1147, in metrics_v

Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 376.37it/s] 


  pk_date_sid   full_date  year  month month_name  day_of_month  day_name  \
0    20140417  2014-04-17  2014      4      April            17  Thursday   
1    20150423  2015-04-23  2015      4      April            23  Thursday   
2    20170413  2017-04-13  2017      4      April            13  Thursday   
3    20210418  2021-04-18  2021      4      April            18    Sunday   

   is_weekday               load_date  
0           1  2025-06-13 03:46:55 PM  
1           1  2025-06-13 03:46:55 PM  
2           1  2025-06-13 03:46:55 PM  
3           0  2025-06-13 03:46:55 PM  


Calculating Metrics: 100%|██████████| 1/1 [00:00<00:00, 616.90it/s] 

Validation results for olist_brazilian_ecommerce_target_star.DIM_DATE:
{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_be_of_type",
        "kwargs": {
          "column": "date_sid",
          "type_": "integer",
          "batch_id": "olist_brazilian_ecommerce_target_star.DIM_DATE_data_source-olist_brazilian_ecommerce_target_star.DIM_DATE_asset"
        },
        "meta": {},
        "id": "5dc22bd2-4f3b-45ba-8de8-9a09830a27d9"
      },
      "result": {},
      "meta": {},
      "exception_info": {
        "exception_traceback": "Traceback (most recent call last):\n  File \"/Users/sqa.k/miniconda3/envs/elt/lib/python3.10/site-packages/great_expectations/validator/validator.py\", line 650, in graph_validate\n    result = expectation.metrics_validate(\n  File \"/Users/sqa.k/miniconda3/envs/elt/lib/python3.10/site-packages/great_expectations/expectations/expectation.py\", line 1147, in metrics_val

In [17]:
context = gx.get_context()

# Query the fact table from GBQ
project_id = "projectm2-aiess"
fact_table_name = "olist_brazilian_ecommerce_target.FCT_ORDERS"
query = f"SELECT * FROM {fact_table_name}"
df_fact_table = read_gbq(query, project_id=project_id)

# Generate unique names for data source and asset
data_source_name = f"{fact_table_name}_data_source"
asset_name = f"{fact_table_name}_asset"

# Add data source
data_source = context.data_sources.add_pandas(name=data_source_name)

# Add DataFrame asset
data_asset = data_source.add_dataframe_asset(name=asset_name)

# Add batch definition
batch_definition_name = fact_table_name
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

# Get the batch and print the first few rows
batch_parameters = {"dataframe": df_fact_table}
batch = batch_definition.get_batch(batch_parameters=batch_parameters)
print(f"Batch for {fact_table_name}:")
print(batch.head(4))

# Add column expectations
schema_fct_orders_expectation_1 = gx.expectations.ExpectColumnToExist(
    column="payment_sid", column_index=0
)
schema_fct_orders_expectation_2 = gx.expectations.ExpectColumnToExist(
    column="review_sid", column_index=1
)
schema_fct_orders_expectation_3 = gx.expectations.ExpectColumnToExist(
    column="item_sid", column_index=2
)
schema_fct_orders_expectation_4 = gx.expectations.ExpectColumnToExist(
    column="customer_sid", column_index=3
)

# Create a new suite for the fact table schema validation
suite_name = "schema_fct_orders_expectation"
suite = gx.ExpectationSuite(name=suite_name)
suite = context.suites.add(suite)

suite.add_expectation(schema_fct_orders_expectation_1)
suite.add_expectation(schema_fct_orders_expectation_2)
suite.add_expectation(schema_fct_orders_expectation_3)
suite.add_expectation(schema_fct_orders_expectation_4)

# Create validation definition
definition_name = "schema_fct_orders_definition"
validation_definition = gx.ValidationDefinition(
    data=batch_definition, suite=suite, name=definition_name
)

# Run validation
validation_results = validation_definition.run(batch_parameters=batch_parameters)
print(f"Validation results for {fact_table_name}:")
print(validation_results)

GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/projectm2-aiess/queries?prettyPrint=false: Access Denied: Table olist-462315:Olist_csv.olist_orders: User does not have permission to query table olist-462315:Olist_csv.olist_orders, or perhaps it does not exist.

In [24]:
context = gx.get_context()
suite_name = "df_delivery_expectation_suite"
suite = gx.ExpectationSuite(name=suite_name)
suite = context.suites.add(suite)

# Add expectations for column existence
suite.add_expectation(
    gx.expectations.ExpectColumnToExist(column="actual_delivery_time")
)
suite.add_expectation(
    gx.expectations.ExpectColumnToExist(column="estimated_delivery_time")
)
suite.add_expectation(
    gx.expectations.ExpectColumnToExist(column="actual_delivery_time_minutes")
)
suite.add_expectation(
    gx.expectations.ExpectColumnToExist(column="estimated_delivery_time_minutes")
)

# Add expectations for column values to be non-null
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="actual_delivery_time")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="estimated_delivery_time")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="actual_delivery_time_minutes")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="estimated_delivery_time_minutes")
)

# Add expectations for column values to be positive
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="actual_delivery_time_minutes", min_value=0
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="estimated_delivery_time_minutes", min_value=0
    )
)

# Add expectations for column data types
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeOfType(
        column="actual_delivery_time", type_="timedelta64[ns]"
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeOfType(
        column="estimated_delivery_time", type_="timedelta64[ns]"
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeOfType(
        column="actual_delivery_time_minutes", type_="float"
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeOfType(
        column="estimated_delivery_time_minutes", type_="float"
    )
)

# Add expectations for column value ranges
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="actual_delivery_time_minutes", min_value=0, max_value=10000
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="estimated_delivery_time_minutes", min_value=0, max_value=10000
    )
)

# Validate the DataFrame
batch_definition = data_asset.add_batch_definition_whole_dataframe("df_delivery_expectation_batch")
batch_parameters = {"dataframe": df_fact_table}
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

validation_definition = gx.ValidationDefinition(
    data=batch, suite=suite, name="df_delivery_validation"
)
validation_results = validation_definition.run(batch_parameters=batch_parameters)
print(validation_results)

NameError: name 'df_fact_table' is not defined